In [ ]:
import numpy as np 
import pandas as pd
import os
import re
import string
from keras_tuner import RandomSearch
import numpy as np 
import random
import pandas as pd 
from keras.layers import (LSTM,Embedding,BatchNormalization,Dense,TimeDistributed,Dropout,Bidirectional,Flatten,GlobalMaxPool1D,GlobalAveragePooling1D,MultiHeadAttention,LayerNormalization,SimpleRNN)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import iplot
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from lime.lime_text import LimeTextExplainer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, 
                          Embedding, 
                          BatchNormalization,
                          Dense, 
                          TimeDistributed, 
                          Dropout, 
                          Bidirectional,
                          Flatten, 
                          GlobalMaxPool1D)
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.utils import pad_sequences
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from keras.optimizers import RMSprop

from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv("spam.csv", delimiter=',',encoding='latin-1')
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.rename(columns = {"v1": "target", "v2": "text"}, inplace = True)
x = ['ham', 'spam']
y = df.groupby("target")["target"].agg("count").values
df['text_len'] = df['text'].apply(lambda x: len(x.split(' ')))

In [ ]:
layout = go.Layout(title={'text':'Proportional Distribution of the Target Variable',
                         'y':0.9,
                         'x':0.5,
                         'xanchor':'center',
                         'yanchor':'top'},
                  template = 'plotly_dark')

fig = go.Figure(data=[go.Bar(
    x = x, y = y,
    text = y, textposition = 'auto',
    marker_color = "slateblue"
)], layout = layout)
fig.show()

In [ ]:
colors = ["slateblue", "darkred"]

fig = go.Figure(data=[go.Pie(labels = df['target'].value_counts().keys(),
                             values = df['target'].value_counts().values,
                             pull = [0, 0.25])])

fig.update_traces(hoverinfo ='label',
                  textinfo ='percent',
                  textfont_size = 20,
                  textposition ='auto',
                  marker=dict(colors=colors,
                              line = dict(color = 'lightgray',
                                          width = 1.5)))
fig.update_layout(title={'text': "Percentages of the Target Values",
                         'y':0.9,
                         'x':0.5,
                         'xanchor': 'center',
                         'yanchor': 'top'},
                  template='plotly_dark')

iplot(fig)

In [ ]:
ham = df[df["target"] == "ham"]["text_len"].value_counts().sort_index()
spam = df[df["target"] == "spam"]["text_len"].value_counts().sort_index()
fig = go.Figure()
fig.add_trace(go.Scatter(x = ham.index, y = ham.values, name = "ham", 
                         fill = "tozeroy"))
fig.add_trace(go.Scatter(x = spam.index, y = spam.values, name="spam",
                        fill = "tozeroy"))
fig.update_layout(title={'text': "Distributions of Target Values",
                         'y':0.9,
                         'x':0.5,
                         'xanchor': 'center',
                         'yanchor': 'top'},
                  template='plotly_dark')
fig.update_xaxes(range=[0, 50])
fig.update_yaxes(range=[0, 450])
fig.show()

In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
df['text'] = df['text'].apply(clean_text)
nltk.download('stopwords')
stop_words = stopwords.words("english")
df["text"] = df["text"].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
stemmer = nltk.SnowballStemmer("english")
df["text"] = df["text"].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))
text = " ".join(i for i in df.text)

wc = WordCloud(background_color = "black", width = 1200, height = 600,
               contour_width = 0, contour_color = "#410F01", max_words = 1000,
               scale = 1, collocations = False, repeat = True, min_font_size = 1)

wc.generate(text)
lb = LabelEncoder()
df["target"] = lb.fit_transform(df["target"])

In [ ]:
X = df['text']
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

vec = CountVectorizer()
vec.fit(X_train)

X_train_dtm = vec.transform(X_train)
X_test_dtm = vec.transform(X_test)
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [ ]:
y_test1 = y_test
X_test1 = X_test
try:
  while y_test1[y_test1 == 0].index[0]:
    index_30 = y_test1[y_test1 == 0].index[0]
    y_test1.drop(index_30, inplace = True)
    X_test1.drop(index_30, inplace = True)
except: pass


In [ ]:
def LSTM_model():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
def RNN_model():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = SimpleRNN(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
def DNN_model():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Dense(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model_rnn = RNN_model()
model_rnn.summary()
model_rnn.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model_dnn = DNN_model()
model_dnn.summary()
model_dnn.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model_lstm = LSTM_model()
model_lstm.summary()
model_lstm.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model_lstm.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
model_rnn.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
model_dnn.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
test_sequences = tok.texts_to_sequences(X_test1)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
y_pred_dnn = model_dnn.predict(test_sequences_matrix)

accr = model_dnn.evaluate(test_sequences_matrix,y_test1)


In [ ]:
test_sequences = tok.texts_to_sequences(X_test1)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
y_pred_lstm = model_lstm.predict(test_sequences_matrix)

accr = model_lstm.evaluate(test_sequences_matrix,y_test1)


In [ ]:
test_sequences = tok.texts_to_sequences(X_test1)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
y_pred_rnn = model_rnn.predict(test_sequences_matrix)

accr = model_rnn.evaluate(test_sequences_matrix,y_test1)

In [ ]:
# 4. Define the Transformer model with tunable hyperparameters
def build_model(hp):
    input_dim = max_words
    output_dim = hp.Int('output_dim', min_value=32, max_value=128, step=32)
    num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2)
    dff = hp.Int('dff', min_value=32, max_value=128, step=32)
    rate = hp.Float('rate', min_value=0.1, max_value=0.5, step=0.1)

    inputs = Input(shape=(max_len,))
    embedding_layer = Embedding(input_dim, output_dim, input_length=max_len)(inputs)
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=output_dim)(embedding_layer, embedding_layer)
    attn_output = LayerNormalization(epsilon=1e-6)(embedding_layer + attn_output)
    ffn_output = Dense(dff, activation='relu')(attn_output)
    ffn_output = Dense(output_dim)(ffn_output)
    ffn_output = LayerNormalization(epsilon=1e-6)(attn_output + ffn_output)
    pooling = GlobalAveragePooling1D()(ffn_output)
    outputs = Dense(1, activation='sigmoid')(pooling)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 5. Set up Keras Tuner and perform hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory='sms_spam_tuner',
    project_name='sms_spam_transformer'
)

tuner.search_space_summary()

tuner.search(sequences_matrix,y_train, epochs=1, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

#6. Get the best model and its hyperparameters

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
history = best_model.fit(sequences_matrix, y_train, batch_size=32, epochs=1, validation_split=0.2)


In [ ]:
test_sequences = tok.texts_to_sequences(X_test1)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
y_pred_transformer = best_model.predict(test_sequences_matrix)
accr = best_model.evaluate(test_sequences_matrix,y_test1)


In [ ]:
count = 0
for i in range(len(y_pred_lstm)):
  val = [0,0,0]
  if y_pred_lstm[i]>=0.5:
    val[0] = 1
  try:
    if y_pred_dnn[i]>=0.5:
      val[1] = 1
  except: pass  
  if y_pred_rnn[i]>=0.5:
    val[2] = 1
  if val.count(1)>=2:
    count = count+1


  

In [ ]:
class_names = ["ham", "spam"]
explainer = LimeTextExplainer(class_names = class_names)
train_df = df

## split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)
val_df.reset_index(drop=True)

## vectorize to tf-idf vectors
tfidf_vc = TfidfVectorizer(min_df = 10, max_features = 100000, analyzer = "word", ngram_range = (1, 2), stop_words = 'english', lowercase = True)
train_vc = tfidf_vc.fit_transform(train_df["text"])
val_vc = tfidf_vc.transform(val_df["text"])
spam_index = y_test1.index[y_test1 == 1]
spam_index = list(spam_index)

In [ ]:

idx = spam_index[11]
model_adv = LogisticRegression(C = 0.5, solver = "sag")
model_adv = model_adv.fit(train_vc, train_df.target)
val_pred = model_adv.predict(val_vc)


val_cv = f1_score(val_df.target, val_pred, average = "binary")
print(val_cv)


c = make_pipeline(tfidf_vc, model_adv)
class_names = ["ham", "spam"]
explainer = LimeTextExplainer(class_names = class_names)
exp = explainer.explain_instance(df["text"][idx], c.predict_proba)
exp.as_list()

In [ ]:
print(idx)
X_test1[idx] = "winner value network custom select receive a reward call claim code valid hour"
test_sequences = tok.texts_to_sequences(X_test1)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
y_pred = model.predict(test_sequences_matrix)
y_pred[11]

In [ ]:
X_test1 = X_test1.str.replace('call', 'phone')
words = X_test1.str.split(expand=True).stack()
word_freq = words.value_counts()

In [ ]:
X_test1 = X_test1.str.replace('txt', 'text')
words = X_test1.str.split(expand=True).stack()
word_freq = words.value_counts()
word_freq

In [ ]:
X_test1 = X_test1.str.replace('å£', ' ')
X_test1 = X_test1.str.replace('optin', 'option')
X_test1 = X_test1.str.replace('mobilee', 'mobile')
X_test1 = X_test1.str.replace('repli', 'reply')
X_test1 = X_test1.str.replace('claim', ' ')
X_test1 = X_test1.str.replace('servic', 'service')
X_test1 = X_test1.str.replace('prize', ' ')
words = X_test1.str.split(expand=True).stack()
word_freq = words.value_counts()
word_freq.head(10)

In [ ]:
X_test1 = X_test1.str.replace('text', ' ')
word_freq.head(20)

In [ ]:
X_test1[1044]="know someon know deluxe call find"
X_test1[683] = "hi i am sue year old lapdanc love sex live im bedroom text sue textoper"